**Breakdown of Scraping Notebook**

0. Import required packages, set variables, etc. Open up an instance of Selenium webdriver.
1. Check for existence of TCN files for the specific year.
    - If they're not there, build TCN files. They need to include the following and be saved in a separate directory.
        - Year
        - Team Name (per NCAA)
        - UID
        - Team Conference (abbreviated as NCAA does)
        - Year-specific Team Number (from URL/href)
    - If they're there, proceed.
2. Loop over the TCN files. Get the data that is available from the team pages.
3. Save the data in year-specific and overall files, both XLSX and CSV.

In [1]:
# Define key functions
def BecomeTeamInUID(teamName):
    return teamName.replace(" (", "-").replace(")", "").replace(".", "").replace(" ", "")
def TeamWithoutParens(teamName):
    return teamName.replace(" (", "-").replace(")", "")
def BecomeUID(teamName, year, sport_abbr):
    newTeamName = BecomeTeamInUID(teamName)
    outputUID = f"{year}.{sport_abbr}.{newTeamName}"
    return outputUID
def TeamsToUIDs(teamList, year, sport_abbr):
    uidList = []
    for team in teamList:
        uidList.append(BecomeUID(team, year, sport_abbr))
    return uidList
def DetectLocationDropRanking(opp, locNote):
    opp = opp.strip()
    # opponent is in one of the following formats. First detect home/away/neutral
    # "@ opp name" or "@ #xy opp name" or "@ #xyopp name"
    if (opp[0] == "@" and opp[1] == " "):
        newOpp = opp[2:]
        location = "Away"
    # "@opp name" or "@#xy opp name" or "@#xyoppname"
    elif (opp[0] == "@" and opp[1] != " "):
        newOpp = opp[1:]
        location = "Away"
    # "opp name" or "oppname" or "#xy oppname" or "#xyoppname"
    elif locNote != "":
        newOpp = opp
        location = "Neutral"
    else:
        newOpp = opp
        location = "Home"
    # Now, newOpp is in one of the following formats:
    # "#xy opp name" or "#xyopp name"
    if (newOpp[0] == "#" and newOpp[1].isnumeric() and newOpp[2].isnumeric()):
        actualOpp = newOpp[3:]
    # "#x opp name" or "#xoppname"
    elif (newOpp[0] == "#" and newOpp[1].isnumeric()):
        actualOpp = newOpp[2:]
    else:
        actualOpp = newOpp
    actualOpp = actualOpp.strip()
    return (actualOpp, location)

def DropRanking(team):
    # team is in the format: "#x team name" or "#xy team name" or #xyteam name. outputs "team name"
    poundIndex = team.find("#")
    spaceIndex = team.find(" ", poundIndex)
    if poundIndex == -1:
        return team
    else:
        return team[:poundIndex] + team[(spaceIndex + 1):]

def ReturnLongestDF(dfList):
    if len(dfList) == 1:
        return dfList[0]
    else:
        longestDF = dfList[0]
        for i in range(len(dfList) - 1):
            if len(dfList[i+1]) > len(longestDF):
                longestDF = dfList[i+1]
        return longestDF

**First step**: Leverage team leaderboard page to get "TCN" (team-conference-number) files that will help in building URLs for the next phase of the program.

In [2]:
# Import statements
import os
import io
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import time
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import datetime # added 2-3-24
pd.options.mode.copy_on_write = True

sport = "Softball"
sport_abbr = "SB"
minYear = 2021
maxYear = 2024
division = "1.0"
years = np.flip(np.arange(minYear, maxYear + 1))
baseURL = "https://stats.ncaa.org/rankings/national_ranking?academic_year=2024.0&division=1.0&ranking_period=88.0&sport_code=WSB&stat_seq=320.0"
tcnOutputPath = os.getcwd() + "\\TCN-Files"
statOutputPath = os.getcwd() + "\\Stat-Files"

In [14]:
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(15)

# Build TCN files.
for year in years:
    # Go to 2024 win-loss percentage page
    driver.get(baseURL)
    
    # Select Academic Year.
    AcadYr = str(year - 1) + "-" + str(year)[2:]
    selectYR = Select(driver.find_element(By.NAME, 'acadyr'))
    selectYR.select_by_visible_text(AcadYr)
    
    # Select maximum number of teams.
    selectNumTeams = Select(driver.find_element(By.NAME, 'rankings_table_length'))
    selectNumTeams.select_by_value("-1")   # This selects the greatest number from the dropdown list.
    time.sleep(2)
    
    # Get team numbers from table
    teamNumbers = []
    table = driver.find_element(By.ID, "rankings_table")
    result = BeautifulSoup(table.get_attribute('innerHTML'), 'lxml')
    rows = result.find_all('tr')
    for row in rows:
        link = row.find('a', href=True)
        if link:
            extension = link['href']
            teamNum = extension[7:]
            teamNumbers.append(teamNum)
    
    WLTableDF = pd.read_html(io.StringIO(driver.page_source))[1]
    
    # Split the team into team and conference
    teams = []
    conferences = []
    for teamConf in WLTableDF["Team"]:
        if teamConf.find("Reclass") != -1:
            break
        splitIndex = teamConf.rfind("(")
        teams.append(TeamWithoutParens(teamConf[:(splitIndex-1)]))
        conferences.append(teamConf[(splitIndex+1):-1])
    uidList = TeamsToUIDs(teams, str(year), sport_abbr)
    
    # Construct TCN Dataframe
    tcnDF = pd.DataFrame({"Year" : [year] * len(teams),
                          "Team" : teams,
                          "Conference" : conferences,
                          "UID" : uidList,
                          "YearTeamNumbers" : teamNumbers})
    tcnDF.to_csv(tcnOutputPath + f"\\{year}_TCN.csv")
    
driver.close()

**Next Step:** Use the TCN files to obtain data from individual team pages.

The data that is helpful for these analyses include the following for each game:
- Date
- Team
- Team UID
- Opponent
- Opponent UID
- Win/Loss/Tie
- Location
- Runs Scored
- Runs Against

In [29]:
# Columns for output DF
dfColumns = ["Year", "Date", "Team", "TeamUID", "TeamConference", "Opponent", "OppUID", "OppConference",
             "Location", "Conference", "Win", "Loss", "Tie", "RunsFor", "RunsAgainst"]

# Year -> End date of regular season/conference tournaments
Year_to_RSEndDate = {2023 : datetime.date(2023, 5, 15), 
                     2022 : datetime.date(2022, 5, 16), 
                     2021 : datetime.date(2021, 5, 17)}

service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(30)

# Create overall DF.
overallDF = pd.DataFrame(columns=dfColumns)

# Loop over years
for year in [2022]:
    tcnDF = pd.read_csv(tcnOutputPath + f"\\{year}_TCN.csv")
    tcnToFixDF = pd.read_csv("TeamsToCorrect2_TCN.csv")
    tcnToFixDF = tcnToFixDF[tcnToFixDF["Year"] == year]
    tcnToFixDF.reset_index(inplace=True, drop=True)
    yearDF = pd.DataFrame(columns=dfColumns)
    # Loop over teams in years
    for teamIndex in range(len(tcnToFixDF)):
        team = tcnToFixDF.loc[teamIndex, "Team"]
        teamYearNum = tcnToFixDF.loc[teamIndex, "YearTeamNumbers"]
        # Go to team homepage then the batting page.
        teamURL = f"https://stats.ncaa.org/teams/{teamYearNum}"
        driver.get(teamURL)
        while True:
            try:
                driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/div/div/nav/ul/li[4]/a")
            except:
                time.sleep(2)
                driver.refresh()
            else:
                break
        htmlToReplace = '</a> <br>'
        replacementHTML = '</a> <br>SPLIT'
        # Go to hitting stats page (Game By Game)
        time.sleep(2)
        driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/div/div/nav/ul/li[4]/a").click()
        while True:
            try:
                driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div/div[1]/div[1]')
            except:
                time.sleep(2)
                driver.refresh()
            else:
                break
        battingTable = driver.find_element(By.ID, f"game_breakdown_div").get_attribute('innerHTML')
        battingTable = battingTable.replace(htmlToReplace, replacementHTML)
        battingDFs = pd.read_html(io.StringIO(battingTable))
        # Some pages don't have the actual stat table as the first entry, we have to pick which one we want.
        battingDF = ReturnLongestDF(battingDFs)
        # Go to pitching stats page
        time.sleep(2)
        driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/div/div/nav[2]/ul/li[2]/a").click()
        while True:
            try:
                driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div/div[1]/div[1]')
            except:
                time.sleep(2)
                driver.refresh()
            else:
                break
        pitchingTable = driver.find_element(By.ID, f"game_breakdown_div").get_attribute('innerHTML')
        pitchingTable = pitchingTable.replace(htmlToReplace, replacementHTML)
        pitchingDFs = pd.read_html(io.StringIO(pitchingTable))
        pitchingDF = ReturnLongestDF(pitchingDFs)
        
        # Get rid of battingDF and pitchingDF rows where there isn't a result.
        battingDF.dropna(subset=['Result'], inplace=True)
        pitchingDF.dropna(subset=['Result'], inplace=True)
        battingDF.reset_index(inplace=True, drop=True)
        pitchingDF.reset_index(inplace=True, drop=True)
        
        # Combine dataframes
        teamDF = pd.DataFrame({"Year" : [year] * len(battingDF),
                                "Date" : battingDF["Date"],
                                "Team": [team] * len(battingDF),
                                "Opponent" : battingDF["Opponent"],
                                "Result" : battingDF["Result"],
                                "RunsFor" : battingDF["R"], 
                                "RunsAgainst" : pitchingDF["R"]})
        # Fill in missing values with 0.
        teamDF = teamDF.fillna(0)
        
        
        # Get rid of summary/canceled game columns + add W/L/T columns
        wins = []
        losses = []
        ties = []
        rowsToDrop = []
        for i in range(len(teamDF)):
            result = teamDF.loc[i, "Result"]
            resultStart = result[:2]
            opp = teamDF.loc[i, "Opponent"]
            if resultStart[:2] == "W ":
                wins.append(1); losses.append(0); ties.append(0);
            elif resultStart[:2] == "L ":
                wins.append(0); losses.append(1); ties.append(0);
            elif resultStart[:2] == "T ":
                wins.append(0); losses.append(0); ties.append(1);
            else:
                rowsToDrop.append(i)
        teamDF.drop(rowsToDrop, axis=0, inplace=True)
        teamDF["Win"] = wins
        teamDF["Loss"] = losses
        teamDF["Tie"] = ties
        teamDF.reset_index(inplace=True, drop=True)
        
        # Split Opponent into Opponent and Location Note
        newOpponents = []
        locationNotes = []
        for i in range(len(teamDF)):
            oldOpponent = teamDF.loc[i, "Opponent"]
            findSplit = oldOpponent.find("SPLIT")
            if findSplit == -1: # it's normal
                newOpponents.append(oldOpponent)
                locationNotes.append("")
            else: # we have a SPLIT
                newOpponents.append(oldOpponent[:(findSplit - 1)])
                locationNotes.append(oldOpponent[(findSplit + 5):])
        teamDF["Opponent"] = newOpponents
        teamDF["LocationNote"] = locationNotes
        
        
        # Clean Up Opponent Column, add location column
        locations = []
        newOpponents = []
        for i in range(len(teamDF)):
            opp = teamDF.loc[i, "Opponent"]
            locNote = teamDF.loc[i, "LocationNote"]
            (actualOpp, location) = DetectLocationDropRanking(opp, locNote)
            locations.append(location)
            newOpponents.append(actualOpp)
        teamDF["Location"] = locations
        teamDF["Opponent"] = newOpponents
        teamDF.reset_index(inplace=True, drop=True)
        
        
        # Add UIDs
        teamDF["TeamUID"] = TeamsToUIDs(teamDF["Team"], year, sport_abbr)
        teamDF["OppUID"] = TeamsToUIDs(teamDF["Opponent"], year, sport_abbr)
        
        # Add Conferences
        teamDF = teamDF.merge(tcnDF[["UID", "Conference"]], left_on="TeamUID", right_on="UID")
        teamDF.rename(columns={"Conference" : "TeamConference"}, inplace=True)
        teamDF.drop(columns=["UID"], inplace=True)
        teamDF = teamDF.merge(tcnDF[["UID", "Conference"]], left_on="OppUID", right_on="UID")
        teamDF.rename(columns={"Conference" : "OppConference"}, inplace=True)
        teamDF.drop(columns=["UID"], inplace=True)
        
        # Add conference column
        teamDF["Conference"] = [1 if (teamDF.loc[j, "TeamConference"] == teamDF.loc[j, "OppConference"])
                                else 0 for j in range(len(teamDF)) ]
        
        # Cut teamDF down to only the columns that it's supposed to have.
        teamDF = teamDF[dfColumns]
        
        # Merge into yearDF
        yearDF = pd.concat([yearDF, teamDF])
        
        # Save every team
        print(f"Team {teamIndex + 1} of {len(tcnToFixDF)} in {year} completed with {len(teamDF)} rows. Now saving...")
        yearDF.to_csv(f"SB_{year}_DPGameByGame_Addendum.csv")
    
    # Save yearly
    print(f"{year} addendum completed.")
    yearDF.to_csv(f"SB_{year}_DPGameByGame_Addendum.csv")

    # Append yearDF and save
    overallDF = pd.concat([overallDF, yearDF])
    overallDF.to_csv(f"SB_{year}to{maxYear}_DPGameByGame_Addendum.csv")
    
# Save Overall File
#overallDF.to_csv(statOutputPath + f"\\SB_{minYear}to{maxYear}_DPGameByGame.csv")

overallDF.to_csv(f"SB_{minYear}to{maxYear}_DPGameByGame_Addendum.csv")

driver.close()

Team 1 of 22 in 2022 completed with 60 rows. Now saving...
Team 2 of 22 in 2022 completed with 63 rows. Now saving...
Team 3 of 22 in 2022 completed with 68 rows. Now saving...
Team 4 of 22 in 2022 completed with 59 rows. Now saving...
Team 5 of 22 in 2022 completed with 59 rows. Now saving...
Team 6 of 22 in 2022 completed with 43 rows. Now saving...
Team 7 of 22 in 2022 completed with 55 rows. Now saving...
Team 8 of 22 in 2022 completed with 60 rows. Now saving...
Team 9 of 22 in 2022 completed with 51 rows. Now saving...
Team 10 of 22 in 2022 completed with 54 rows. Now saving...
Team 11 of 22 in 2022 completed with 57 rows. Now saving...
Team 12 of 22 in 2022 completed with 47 rows. Now saving...
Team 13 of 22 in 2022 completed with 45 rows. Now saving...
Team 14 of 22 in 2022 completed with 54 rows. Now saving...
Team 15 of 22 in 2022 completed with 52 rows. Now saving...
Team 16 of 22 in 2022 completed with 45 rows. Now saving...
Team 17 of 22 in 2022 completed with 50 rows. Now

Last Step: Cleaning Data

We want to get the "Date" column in the proper form (without any (1) to indicate the first game of a doubleheader and (2) to indicate the second game of a doubleheader) and the 

In [12]:
yearDFList = []
for year in years:
    yearDF = pd.read_csv(statOutputPath + f"\\SB_{year}_DPGameByGame.csv")
    yearDF["Date"] = [yearDF.loc[i, "Date"].replace("(1)", "").replace("(2)", "").replace("(3)", "") for i in range(len(yearDF))]
    yearDF["Date"] = pd.to_datetime(yearDF["Date"])
    yearDF["RunsFor"] = [int(float(yearDF.loc[i, "RunsFor"].replace("/",""))) for i in range(len(yearDF))]
    yearDF["RunsAgainst"] = [int(float(yearDF.loc[i, "RunsAgainst"].replace("/",""))) for i in range(len(yearDF))]
    yearDF.to_csv(statOutputPath + f"\\SB_{year}_DPGameByGame.csv")
    yearDFList.append(yearDF)
yearsDF = pd.concat(yearDFList)
yearsDF.to_csv(statOutputPath + f"\\SB_{minYear}to{maxYear}_DPGameByGame.csv")